# Sonecule: DiscreteCallbackPMS - DiscretePMS Sonecule using a Callback function for flexible mapping

This notebook introduces and demonstrates usage of the DiscreteCallbackPMS sonecule.
* The sonecule spawns synth for each data point in a data set.
* The synth offers several parameters that can set at init time
* Specifically the parameters are:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
  * attack time
  * duration of the event
  * release time
* Different from the DiscretePMS, where the mapping is specified by a mapping specification dictionary that is parsed column by column, this sonecule calls a callback function for each row, providing the row vector as an argument. 
* this allows highly flexible mappings and even things that are impossible with the non-callback Sonecule, such as suppressing sounds if certain conditions throughout channels apply.
* Definition of a callback function is a daunting task for non-programmers. Therefore the sonecule comes with a create_callback_template method, that delivers a function that can be copied and pasted in a jupyter notebook cell (or your IDE), as starting point for own sonification designs.

## Data Preparation

In [ ]:
# headers and imports for the demo
import sonecules as sn
import sc3nb as scn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt
import time

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
df = dataframes["building"]
df.plot(lw=0.2);

In [ ]:
sns.pairplot(df.iloc[2000:, 10:], height=0.8, plot_kws={"s": 3});

## Usage / Demo Snippets

Let's create a ContinuousCallbackPMS using a callback function for mapping.

First we need to import the sonecule

In [ ]:
from sonecules.scoresyn import DiscreteCallbackPMS

The following cell shows all steps usually used to sonify the data, i.e., 
- creation of the sonecule
- cleaning the time line
- defining the callback function (here named callback_fn, but any name is fine)
- creating the sonification using the sonecules schedule function
- starting your sonification using the sonecule start function

For the example we start with one week =7*24 hourly measurments of the building data set

In [ ]:
# create the sonecule
scb = DiscreteCallbackPMS(df.iloc[:60*24, 10:])  # one week, omit the first 6 columns

# clear the timeline
ctx.timeline.reset()

# this is just a shortcut to the mapcol function which maps the columns data 
# mapcol(r, feature, column_mins, column_maxs, target_min, target_max)
mapcol = scb.mapcol

# define the callback function as needed/wanted, for example
def callback_fn(r, cmi, cma, pp):
    pp['onset']     = mapcol(r, 'humidity', cmi, cma, 0, 9)
    pp['freq']      = pam.midicps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']       = pam.dbamp(mapcol(r, 'solar_radiation', cmi, cma, -40, -20)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    return pp # it returns the mapping dictionary

# sonify the data using the above callback function
scb.schedule(at=0, callback_fn=callback_fn)

# start the playback 
scb.start(rate=1);

- play with the definition of the callback_fn and execute again to explore the data
- to see available column names use

In [ ]:
# to check the sc3 scope:
# ctx.backend.sc.lang.cmd("s.scope")

In [ ]:
scb.data.columns

- control the duration and data slice.
- Once you want to keep the callback function fixed there is no need to reexecute
- Here some other data sonified with the same callback function

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[30*24:60*24, 9:])  # the second month of data in 10 seconds
ctx.timeline.reset()
scb.schedule(at=0, callback_fn=callback_fn).start();

If wanted we can have a sonification that specifies or uses more parameters of the synths.
If you use the default synth ("dcbpms"), its parameters are

In [ ]:
scb.syn.params

As of now synths do not come with given bounds, but it is useful to design and specify them and later sonecules can pick from a library of curated synth definitions with suitable bounds.
Let's define a more complete mapping

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[::2, 9:])  # all data, every 2nd row (>6m) in 15 seconds

# here a mapping function that specifies all synth parameters 
def callback_fn(r, cmi, cma, pp):
    pp['onset']     = mapcol(r, 'humidity', cmi, cma, 0, 9)
    pp['freq']      = pam.midicps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']       = pam.dbamp(mapcol(r, 'solar_radiation', cmi, cma, -30, 0)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['sharp']     = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibir']     = 0.05
    pp['dur']       = 0.1
    pp['rel']       = pp['dur']
    return pp # it returns the mapping dictionary
ctx.timeline.reset()
scb.schedule(at=0.5, callback_fn=callback_fn).start();

- As this is all executed in real-time, you will probably experience limits of your system.
- Sonecules will issue late warnings if there are processing-based delays
- in such cases, a non-real-time rendering is always an option to compute a guaranteed correct sonification. This will be demoed elsewhere. The plan is to integrate this as an option into all sonecules.


In [ ]:
ctx.stop()  # execute this in case an error occurs and synths keep on playing

Now usually users like to use their own developed synth. This can be done as follows:

In [ ]:
# create your synths
ctx.synths.add_synth_def("mysyn", r"""{ | out=0, freq=400, dur=0.1, amp=0.5, rq=0.1, pan=0 |
    var noise = WhiteNoise.ar(amp);
    var filt = BPF.ar(noise, freq, rq);
    var env = Line.kr(1, 0, dur, doneAction: 2);
    Out.ar(out, Pan2.ar(filt, pan, env));
}""")

# create sonecule
scb = DiscreteCallbackPMS(df.iloc[::2, 9:], "mysyn")  # all data, every 2nd row (>6m) in 15 seconds

# here a mapping function that specifies all synth parameters 
def callback_fn(r, cmi, cma, pp):
    pp['onset']     = mapcol(r, 'humidity', cmi, cma, 0, 4)
    pp['freq']      = pam.midicps(mapcol(r, 'temperature', cmi, cma, 58, 82))
    pp['amp']       = pam.dbamp(mapcol(r, 'solar_radiation', cmi, cma, -10, 10)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['dur']       = 0.1
    pp['rq']       = 0.1
    return pp # it returns the mapping dictionary

ctx.timeline.reset()

scb.schedule(at=0.5, callback_fn=callback_fn).start();

Now let the sonecule propose a mapping as starting point for own experiments.

- Calling `create_callback_template()` 
- composes and then prints the python code string.
- The output can be copied into a notebook cell and adapted as needed.
- On execution it defines the callback function cbfn().
- The sonification can be rendered as you go until satisfactory.

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[:7*24, 8:], "mysyn")  # first week but no weekday features
fnstr = scb.create_callback_template(auto_assign=True, duration=8)

The printout shows python code 
- that you can simply copy and paste into a new code cell
- ATTENTION: note that the code might not be delivering useful sound as such
  - for instance: 'amp' has no default bounds, so mapping might be from 0 to 0 - not audible
  - for instance: 'onset' may be mapped from a feature in which data don't spread nicely...
  - so best is to check the mapping, adjust so that it makes sense 
- modify the cell as you see fit, e.g. remove rows, or set the right hand side to a constant as preferred
- once satisfied, you can execute the function.
- Note that the code line with the schedule is commented out: the variable name of the sonecule class can't be guessed, so this has to be manually adjusted


In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are:'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    pp['onset']    	 = mapcol(r, 'temperature', cmi, cma, 0.00, 8.00)
    pp['freq']     	 = mapcol(r, 'am_pm', cmi, cma, 300.00, 600.00)
    pp['dur']      	 = mapcol(r, 'temperature', cmi, cma, 0.08, 0.15)
    pp['amp']      	 = mapcol(r, 'humidity', cmi, cma, 0.38, 0.75)
    pp['rq']       	 = mapcol(r, 'solar_radiation', cmi, cma, 0.01, 0.05)
    pp['pan']      	 = mapcol(r, 'wind_speed', cmi, cma, 0.00, 0.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, callback_fn=cbfn).start(rate=1);

Let's create another sonification for the mesonic default synth "s1"

In [ ]:
scb = DiscreteCallbackPMS(df.iloc[:60*24, 10:], "s1")  # for 2 months data
fnstr = scb.create_callback_template(auto_assign=True, duration=8)

In [ ]:
# this cell just is copied the output of the previous cell for a test...
def cbfn(r, cmi, cma, pp):
    # columns are:'temperature' 'humidity' 'solar_radiation' 'wind_speed' 
    # 'hc_wb_electrical' 'hc_wb_cold_water' 'hc_wb_hot_water' 
    pp['onset']    	 = mapcol(r, 'temperature', cmi, cma, 0.00, 8.00)
    pp['freq']     	 = mapcol(r, 'humidity', cmi, cma, 300.00, 600.00)
    pp['amp']      	 = mapcol(r, 'solar_radiation', cmi, cma, 0.23, 0.45)
    pp['num']      	 = mapcol(r, 'wind_speed', cmi, cma, 3.00, 6.00)
    pp['pan']      	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 0.00, 0.00)
    pp['dur']      	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.30, 0.60)
    pp['att']      	 = mapcol(r, 'temperature', cmi, cma, 0.01, 0.01)
    pp['curve']    	 = mapcol(r, 'humidity', cmi, cma, -1.50, -3.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, callback_fn=cbfn).start(rate=1);

And finally here is a hand-crafted mapping, from modifying the code

In [ ]:
ctx.backend.stop(ctx)